In [1]:
# import libraries
import torch
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import MNIST
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt

In [2]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 28, 28)):

    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()

In [3]:
def make_grad_hook():
    '''
    Function to keep track of gradients for visualization purposes, 
    which fills the grads list when using model.apply(grad_hook).
    '''
    grads = []
    def grad_hook(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            grads.append(m.weight.grad)
    return grads, grad_hook

In [4]:
# noise generation
def get_noise(n_samples, z_dim, device='cpu'):
    
    return torch.randn(n_samples, z_dim, device=device)

In [5]:
# Generator
class Generator(nn.Module):
   
    def __init__(self, z_dim=64, im_chan=1, hidden_dim=64):
        super(Generator, self).__init__()
        self.z_dim = z_dim
        # Build the neural network
        self.gen = nn.Sequential(
            self.make_gen_block(z_dim, hidden_dim * 4),
            self.make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size=4, stride=1),
            self.make_gen_block(hidden_dim * 2, hidden_dim),
            self.make_gen_block(hidden_dim, im_chan, kernel_size=4, final_layer=True),
        )

    def make_gen_block(self, input_channels, output_channels, kernel_size=3, stride=2, final_layer=False):
     
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True),
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride),
                nn.Tanh(),
            )

    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor,
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        x = noise.view(len(noise), self.z_dim, 1, 1)
        return self.gen(x)

In [6]:
# Critic (in W-GANS we discriminators are called critics because they give you scores)
class Critic(nn.Module):
    def __init__(self, im_chan=1, hidden_dim = 64):
        super(Critic, self).__init__()
        self.critic = nn.Sequential(
            self.make_critic_block(im_chan, hidden_dim),
            self.make_critic_block(hidden_dim, hidden_dim*2),
            self.make_critic_block(hidden_dim*2, 1, final_layer=True)
        )
        
        
    def make_critic_block(self, input_channels, output_channels, kernel_size=4, stride=2, final_layer=False):
        if not final_layer:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.LeakyReLU(0.2, inplace=True)
            )
        else:
            return nn.Sequential(
                nn.Conv2d(input_channels, output_channels, kernel_size, stride),
            )
            
    def forward(self, image):
        # returns a 1D tensor of image
        critic_pred = self.critic(image)
        return critic_pred.view(len(critic_pred), -1)

In [7]:
## Training initialization
n_epochs = 100
z_dim = 64
display_step = 500
batch_size = 128
lr = 0.0002
beta_1 = 0.5
beta_2 = 0.999
c_lambda = 10
crit_repeats = 5
device = 'cuda'

transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,)),
])

dataloader = DataLoader(
    MNIST('.', download=False, transform=transform),
    batch_size=batch_size,
    shuffle=True)

In [8]:
## initializing generator, critic, and optimizers.
gen = Generator().to(device)
gen_opt = torch.optim.Adam(gen.parameters(), lr = lr, betas=(beta_1, beta_2))

crit = Critic().to(device)
crit_opt = torch.optim.Adam(crit.parameters(), lr=lr, betas=(beta_1, beta_2))

def weights_init(m):
    if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
    if isinstance(m, nn.BatchNorm2d):
        torch.nn.init.normal_(m.weight, 0.0, 0.02)
        torch.nn.init.constant_(m.bias, 0)
gen = gen.apply(weights_init)
crit = crit.apply(weights_init)

## Gradient penalty

In [9]:
def get_gradient(crit, real, fake, epsilon):
    ## mix images
    mixed_images = real * epsilon + fake * (1 - epsilon)
    
    ## critic's score
    mixed_scores = crit(mixed_images)
    
    # Take the gradient of the scores with respect to the images
    gradient = torch.autograd.grad(inputs=mixed_images, outputs=mixed_scores,
                                   grad_outputs=torch.ones_like(mixed_scores),
                                   create_graph=True, retain_graph=True)[0]
    return gradient
    

In [10]:
def gradient_penalty(gradient):
    ## flatten gradients so each row captures one image
    gradient = gradient.veiw(len(gradient), -1)
    gradient_norm = gradient.norm(2, dim=1)
    penalty = torch.mean((gradient_norm - 1)**2)
    return penalty

In [12]:
## losses
def get_gen_loss(critic_fake_pred):
    gen_loss = -torch.mean(critic_fake_pred)

def get_critic_loss(critic_fake_pred, critic_real_pred, gp, c_lambda):
    critic_loss = torch.mean(critic_fake_pred) - torch.mean(critic_real_pred) + c_lambda*gp

In [13]:
## actual training
from statistics import mean


cur_step = 0
generator_losses = []
critic_losses = []

for epcoh in range(n_epochs):
    # load data
    for real,_ in tqdm(dataloader):
        cur_batch_size = len(real)
        real = real.to(device)
        
        mean_iteration_critic_loss = 0
        for _ in range(crit_repeats):
        
            ## critic 
            crit_opt.zero_grad()
            fake_noise = get_noise(cur_batch_size, z_dim, device=device)
            fake= gen(fake_noise)
            crit_fake_pred = crit(fake.detach())
            crit_real_pred = crit(real)
            
            epsilon = torch.rand(len(real),1,1,1, device=device, requires_grad=True)
            gradient = get_gradient(crit, real, fake.detach(), epsilon)
            gp = gradient_penalty(gradient)
            crit_loss = get_critic_loss(crit_fake_pred, crit_real_pred, gp, c_lambda)
            
            # Keep track of the average critic loss in this batch
            mean_iteration_critic_loss += crit_loss.item() / crit_repeats
            
            #update gradients 
            crit_loss.backward(retain_graph=True)
            #update optimizer
            crit_opt.step()
        
    critic_losses += [mean_iteration_critic_loss]
    
    ## generator
    gen_opt.zero_grad()
    fake_noise_2 = get_noise(cur_batch_size, z_dim, device=device)
    fake_2 = gen(fake_noise_2)
    crit_fake_pred = crit(fake_2)
    gen_loss = get_gen_loss(crit_fake_pred)
    gen_loss.backward()
    gen_opt.step()
    
    # Keep track of the average generator loss in this batch
    generator_losses += [gen_loss.item()]
    
    ## visualization
    if cur_step % display_step == 0 and cur_step > 0:
        gen_mean = sum(generator_losses[-display_step:]) / display_step
        crit_mean = sum(critic_losses[-display_step:]) / display_step
        print(f"Epoch {epoch}, step {cur_step}: Generator loss: {gen_mean}, critic loss: {crit_mean}")
        show_tensor_images(fake)
        show_tensor_images(real)
        step_bins = 20
        num_examples = (len(generator_losses) // step_bins) * step_bins
        plt.plot(range(num_examples // step_bins),
                 torch.Tensor(generator_losses[:num_examples]).view(-1, step_bins).mean(1),
                 label="Generator Loss")
        plt.plot(range(num_examples // step_bins),
                 torch.Tensor(critic_losses[:num_examples]).view(-1, step_bins).mean(1),
                 label="Critic Loss")
        plt.legend()
        plt.show()
    cur_step +=1
        
        
        

  0%|          | 0/469 [00:00<?, ?it/s]

RuntimeError: CUDA error: unspecified launch failure
CUDA kernel errors might be asynchronously reported at some other API call,so the stacktrace below might be incorrect.
For debugging consider passing CUDA_LAUNCH_BLOCKING=1.